In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
#export
from telegram.ext import Updater, InlineQueryHandler, CommandHandler
import pandas as pd
import requests
import re
import lib
import logging
from telegram.ext import Updater, CommandHandler, MessageHandler, Filters

In [3]:
# x = lib.DatabaseIO().read_data('triggers')
# x.head()

,response
trigger,
nick,Nick is niet bereikbaar tussen 9:00 en 17:00
stijn,De Blockstijn
henry,Tot uw dienst!
probleem,https://www.youtube.com/watch?v=x9RsC_zZEfs
herny,Are you herny?


In [3]:
#export
"""
Simple Bot to reply to Telegram messages.

First, a few handler functions are defined. Then, those functions are passed to
the Dispatcher and registered at their respective places.
Then, the bot is started and runs until we press Ctrl-C on the command line.
"""

api_token = lib.config['telegram']['api_token']

# Enable logging
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

class HenryBot:
    def __init__(self):
        self.db_conn = lib.DatabaseIO()
        self.triggers = self.db_conn.read_data('triggers')
    
    @staticmethod
    def start(update, context):
        """Send a message when the command /start is issued."""
        update.message.reply_text('Hi! I am HenryBot and you\'ve triggered me. You can add new triggers using /add. Get more information using /help')
    
    def update_triggers(self):
        self.db_conn.write_data(self.triggers, 'triggers')
    
    def add(self, update, context):
        try:
            value = update.messagetext.split(' ', 1)[1]
            trigger, response = map(strip, value.split(':', 1))
            self.triggers.loc[trigger] = [response]
            self.update_triggers()
            update.message.reply_text(f'\'{trigger}\' added to the triggers')
        except IndexError:
            update.message.reply_text(f"No correctly formatted trigger found. Add a new trigger by typing /add <trigger>:<response>!")

    def delete(self, update, context):
        try:
            trigger = update.message.text.split(' ', 1)[1]
            self.triggers = self.triggers.drop(trigger)
            self.update_triggers()
            update.message.reply_text(f'\'{trigger}\' deleted from triggers.')

        except IndexError:
            update.message.reply_text(f'Please add a trigger to remove. Format is /delete <trigger>')

    def get_triggers(self, update, context):
        message = ''
        update.message.reply_text('\n'.join(self.triggers.index.tolist()))
#         for trigger, row in self.triggers.iterrows():
#             message += f'{trigger}: {row.response}\n'
#         update.message.reply_text(message)         

    def triggered(self, update, context):
        """Echo the user message."""
        text = update.message.text
        user = update.message.from_user
        message = ''
        
        # Respond to 'I am'
        matches = re.search(r'ik ben (\w+)', text, re.I)
        if matches is not None:
            message += f'Hoi {matches.group(1)}, ik ben HenryBot\n'

        # Respond to added triggers
        for trigger, row in self.triggers.iterrows():
            response = row['response']
            regex = r'\b' + trigger + r'\b|\A' + trigger + r'\b '
            if re.search(regex, text, re.I):
                message += self.triggers.loc[trigger, 'response'] + '\n'
                
        message = message.format(user=user.first_name)
        if message:
            update.message.reply_text(message)

    @staticmethod
    def help(update, context):
        """Send a message when the command /help is issued."""
        update.message.reply_text('Help!')


def error(update, context):
    """Log Errors caused by Updates."""
    logger.warning('Update "%s" caused error "%s"', update, context.error)


def main():
    """Start the bot."""
    # Create the Updater and pass it your bot's token.
    # Make sure to set use_context=True to use the new context based callbacks
    # Post version 12 this will no longer be necessary
    henry_bot = HenryBot()
    updater = Updater(api_token, use_context=True)

    # Get the dispatcher to register handlers
    dp = updater.dispatcher

    # on different commands - answer in Telegram
    dp.add_handler(CommandHandler("start", henry_bot.start))
    dp.add_handler(CommandHandler("add", henry_bot.add))
    dp.add_handler(CommandHandler("delete", henry_bot.delete))
    dp.add_handler(CommandHandler("triggers", henry_bot.get_triggers))
    dp.add_handler(CommandHandler("help", henry_bot.help))

    # on noncommand i.e message - respond appropriately to the message on Telegram
    dp.add_handler(MessageHandler(Filters.text, henry_bot.triggered))

    # log all errors
    dp.add_error_handler(error)

    # Start the Bot
    updater.start_polling()
    updater.idle()

In [ ]:
main()

2019-05-19 17:42:59,987 - lib - INFO - Reading data from triggers...
2019-05-19 17:43:02,792 - lib - INFO - Done.
2019-05-19 17:43:29,886 - telegram.ext.dispatcher - WARNING - A TelegramError was raised while processing the Update
2019-05-19 17:43:29,889 - __main__ - WARNING - Update "{'update_id': 365180618, 'message': {'message_id': 63, 'date': 1558280582, 'chat': {'id': 6054587, 'type': 'private', 'username': 'Bauke', 'first_name': 'Bauke', 'last_name': 'Brenninkmeijer'}, 'text': '/triggers', 'entities': [{'type': 'bot_command', 'offset': 0, 'length': 9}], 'caption_entities': [], 'photo': [], 'new_chat_members': [], 'new_chat_photo': [], 'delete_chat_photo': False, 'group_chat_created': False, 'supergroup_chat_created': False, 'channel_chat_created': False, 'from': {'id': 6054587, 'first_name': 'Bauke', 'is_bot': False, 'last_name': 'Brenninkmeijer', 'username': 'Bauke', 'language_code': 'en'}}, '_effective_user': {'id': 6054587, 'first_name': 'Bauke', 'is_bot': False, 'last_name': 

In [37]:
a = ['ATTACHMENT_TYPES', 'MESSAGE_TYPES', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__metaclass__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_effective_attachment', '_id_attrs', '_parse_html', '_parse_markdown', '_quote', 'animation', 'audio', 'author_signature', 'bot', 'caption', 'caption_entities', 'caption_html', 'caption_html_urled', 'caption_markdown', 'caption_markdown_urled', 'channel_chat_created', 'chat', 'chat_id', 'connected_website', 'contact', 'date', 'de_json', 'delete', 'delete_chat_photo', 'document', 'edit_caption', 'edit_date', 'edit_media', 'edit_reply_markup', 'edit_text', 'effective_attachment', 'entities', 'forward', 'forward_date', 'forward_from', 'forward_from_chat', 'forward_from_message_id', 'forward_signature', 'from_user', 'game', 'group_chat_created', 'invoice', 'left_chat_member', 'link', 'location', 'media_group_id', 'message_id', 'migrate_from_chat_id', 'migrate_to_chat_id', 'new_chat_members', 'new_chat_photo', 'new_chat_title', 'parse_caption_entities', 'parse_caption_entity', 'parse_entities', 'parse_entity', 'passport_data', 'photo', 'pinned_message', 'reply_animation', 'reply_audio', 'reply_contact', 'reply_document', 'reply_html', 'reply_location', 'reply_markdown', 'reply_media_group', 'reply_photo', 'reply_sticker', 'reply_text', 'reply_to_message', 'reply_venue', 'reply_video', 'reply_video_note', 'reply_voice', 'sticker', 'successful_payment', 'supergroup_chat_created', 'text', 'text_html', 'text_html_urled', 'text_markdown', 'text_markdown_urled', 'to_dict', 'to_json', 'venue', 'video', 'video_note', 'voice']
{'id': 6054587, 'first_name': 'Bauke', 'is_bot': False, 'last_name': 'Brenninkmeijer', 'username': 'Bauke', 'language_code': 'en'}
a

['ATTACHMENT_TYPES',
 'MESSAGE_TYPES',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__metaclass__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_effective_attachment',
 '_id_attrs',
 '_parse_html',
 '_parse_markdown',
 '_quote',
 'animation',
 'audio',
 'author_signature',
 'bot',
 'caption',
 'caption_entities',
 'caption_html',
 'caption_html_urled',
 'caption_markdown',
 'caption_markdown_urled',
 'channel_chat_created',
 'chat',
 'chat_id',
 'connected_website',
 'contact',
 'date',
 'de_json',
 'delete',
 'delete_chat_photo',
 'document',
 'edit_caption',
 'edit_date',
 'edit_media',
 'edit_reply_markup',
 'edit_text',
 'effective_attachment',
 'entities',
 'forward',
 'forward_date',


In [ ]:
#export
if __name__ == '__main__':
    main()

In [45]:
!python notebook2script.py bot.ipynb   

Converted bot.ipynb to bot.py
